In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
import random
from numpy.random import rand
from IPython.display import display
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import chi2, mutual_info_classif, f_classif, SelectKBest
from sklearn.model_selection import train_test_split, KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.utils.validation import column_or_1d
from sklearn.utils import resample
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report

random_state = 42
import warnings

# Suppress FutureWarning
warnings.filterwarnings("ignore", category=FutureWarning)

In [3]:
#Normally we have to import the dataset
data_url = 'Datasets/Ovarian_cancer.csv'
data = pd.read_csv(data_url)
#data.head(4)

In [4]:
data.shape #just to check the number of instances and features/variables our dataset have

(253, 15155)

In [5]:
#data.isnull().sum() #We check to see if our datasets has any missing values

In [6]:
#lets check the counts of the class types in our datasets
print("We have two classes: ", data['Class'].unique())
data.value_counts('Class')
# This tells us we have 162 instances of type cancer and 91 of type nirmal. Thus we need to do data imbalance handling

We have two classes:  ['Normal' 'Cancer']


Class
Cancer    162
Normal     91
dtype: int64

In [7]:
#machine learning algorithms cannot work with categoricla values, thus he has to label out target to numeric values
label_encoder = LabelEncoder()
data['Class'] = label_encoder.fit_transform(data['Class'])
data.Class.unique()

array([1, 0])

In [8]:
data.Class.value_counts() #We see that the cancer class [0] is the majority class with 162 counts of samoples

0    162
1     91
Name: Class, dtype: int64

In [9]:
data2 = data.copy()#we make a copy of our data for future references

In [10]:
#Now let us handle the imbalancenature of our dataset
#we first haave to separate the two classes
majority_class_cancer = data[data['Class'] == 0]
minority_class_normal = data[data['Class'] == 1]

print("confirming to ensure its 162 samples in the majority class cancer: ",len(majority_class_cancer.Class))
print("confirming to ensure its 91 samples in the minority class cancer: ",len(minority_class_normal.Class))

confirming to ensure its 162 samples in the majority class cancer:  162
confirming to ensure its 91 samples in the minority class cancer:  91


In [11]:
# Upsample minority class to equal majority class
minority_upsample = resample(minority_class_normal, replace = True, n_samples = len(majority_class_cancer), random_state = random_state)
# Combine majority class with upsampled minority class
data_combined = pd.concat([majority_class_cancer, minority_upsample])
print("Checking the shape of our data once more: \n", data_combined.Class.value_counts())

Checking the shape of our data once more: 
 0    162
1    162
Name: Class, dtype: int64


In [12]:
# Display summary statistics to check the range of values so as to know if you need scaling
#summary_stats = data.describe()
#print(summary_stats)

# Check the minimum and maximum values for each feature
min_values = data.min()
if min_values.nunique() == 0.0:
    print("All elements have the same min value.")
else:
    print("Not all elements have the same min value.")

max_values = data.max()
if max_values.nunique() == 1.0:
    print("All elements have the same max value.")
else:
    print("Not all elements have the same max value.")
#from the output we need to perform a minmax scaling to make sure all our values are withing -1 and 1

Not all elements have the same min value.
All elements have the same max value.


In [13]:
#Now we split our dataframe into features and target
features = data.drop(['Class'], axis = 1) #dropping the target column
target = data.loc[:,['Class']] #picking only the target column
#features.head(3)

In [14]:
min_max_scaler = MinMaxScaler(feature_range = (0,1))
data_scaled = min_max_scaler.fit_transform(features)
data_scaled = pd.DataFrame(data_scaled, columns = features.columns)
#data_scaled.head(2)

In [15]:
#SELECTION WITH CHI-SQUARE TEST
data_chi = SelectKBest(score_func = chi2)
data_chi.fit(data_scaled, target)

Table_score = pd.DataFrame(data_chi.scores_)
Table_names = pd.DataFrame(data2.columns)
chi_table = pd.concat([Table_names, Table_score], axis = 1)
chi_table.columns = ['Feature_Names', 'Scores']
chi_table.sort_values(by = 'Scores', ascending = False, inplace = True)
#display(chi_table)
#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)

In [16]:
#SELECTION WITH MUTUAL INFORMATION
data_mi = SelectKBest(score_func = mutual_info_classif)
target_mi = target.copy()
#target_mi = target.ravel()

target_mi = column_or_1d(target_mi)

data_mi.fit(data_scaled, target_mi)

Table_score = pd.DataFrame(data_mi.scores_)
Table_names = pd.DataFrame(data2.columns)
mi_table = pd.concat([Table_names, Table_score], axis = 1)
mi_table.columns = ['Feature_Names', 'Scores']
mi_table.sort_values(by = 'Scores', ascending = False, inplace = True)
#display(mi_table)
#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)

In [19]:
'''Now from our methodology, we aim to use 100, 200, 300, 400, and 500 features where the KNN will
automnatically select the feature subset with better accuracy before paissing to the SSA'''
#we will roughly select 1000 top features from each of the filter methods
Top_1000_chi = chi_table.head(1000)
Top_1000_mi = mi_table.head(1000)

#Now we merge the two top 1000 features from each filter method to remove duplicates. The scoring is again done automatically.
merged_table = pd.concat([Top_1000_chi, Top_1000_mi], axis = 0)
merged_table.shape
#display(merged_table)

(2000, 2)

In [20]:
#we again ensure the features are ranked based on scores
merged_table.sort_values(by = 'Scores', ascending = False, inplace = True)
#display(merged_table)

In [21]:
# Check if any rows have the same value in the specified column
duplicate_values = merged_table.duplicated(subset=['Feature_Names'], keep=False)
# Filter the DataFrame to show only the rows with duplicate values
rows_with_duplicates = merged_table[duplicate_values]
#print(rows_with_duplicates)

In [22]:
#We realised that there are duplicates of features in the dataframe, thus we wil drop the one with a lower rank
# Drop duplicates based on the specified column, keeping the first occurrence
drop_duplicate = merged_table.drop_duplicates(subset = ['Feature_Names'], keep = 'first')
'''This will drop one of the rows that came lower in the DataFrame based on the specified
column name and keep only the first occurrence of the duplicate values''';
drop_duplicate.shape

(1293, 2)

In [41]:
#Now we select top 100 
Top_100 = drop_duplicate.head(100)
Top_100_features = Top_100['Feature_Names'].tolist()
Top_100_data = data2.loc[:,Top_100_features]

#Now we select top 200 
Top_200 = drop_duplicate.head(200)
Top_200_features = Top_200['Feature_Names'].tolist()
Top_200_data = data2.loc[:,Top_200_features]

#Now we select top 300 
Top_300 = drop_duplicate.head(300)
Top_300_features = Top_300['Feature_Names'].tolist()
Top_300_data = data2.loc[:,Top_300_features]

#Now we select top 400 
Top_400 = drop_duplicate.head(400)
Top_400_features = Top_400['Feature_Names'].tolist()
Top_400_data = data2.loc[:,Top_400_features]

#Now we select top 500 
Top_500 = drop_duplicate.head(500)
Top_500_features = Top_500['Feature_Names'].tolist()
Top_500_data = data2.loc[:,Top_500_features]

In [42]:
Top_100_data.shape[1] #sample to look at how it is

100

In [43]:
target = np.ravel(target) #converting our target variable into a numpy array
#target = target.values

In [44]:
data_sizes = [Top_100_data.values, Top_200_data.values, Top_300_data.values,Top_400_data.values,Top_500_data.values]
best_size = []

for data_size in data_sizes:
    kf = KFold(n_splits=10, shuffle=True, random_state=random_state)
    
    for train_index, test_index in kf.split(data_size):
        X_train, X_test = data_size[train_index], data_size[test_index]
        y_train, y_test = target[train_index], target[test_index]

        selection_model = KNeighborsClassifier(n_neighbors = 5)
        selection_model.fit(X_train, y_train)
        y_pred = selection_model.predict(X_test)
        error_rate = 1 - (accuracy_score(y_test, y_pred))
        
    best_size.append(round(error_rate,2))
    #print(f'The error rate of {data_size.shape[1]} top features is: {round(error_rate,2)}, while the accuracy id {1- (round(error_rate,2))}%')
    #print('\n')
print("Error rates of the 5 splits", best_size)

#we auotmatically select the best subset size based on the error rate value and use it in our SSA
error_index = best_size.index(min(best_size)) #find the min value in best_size ie.,min error rate = best acccuracy
selected_Top_data = data_sizes[error_index] # Access the corresponding element in data_sizes using the index of min value of best_size

print('Index of best split: ', error_index) #printing for debugging
print('Shape of selected split: ', selected_Top_data.shape) #printing for debugging



kf = KFold(n_splits=10, shuffle=True, random_state=random_state)
#Because we are using the same value of k and random state, we will always get same train-test split and hence, same result

for train_index, test_index in kf.split(selected_Top_data):
    x_train, x_test = selected_Top_data[train_index], selected_Top_data[test_index]
    y_train, y_test = target[train_index], target[test_index]

fold = {'xt': x_train, 'yt': y_train, 'xv': x_test, 'yv': y_test} #Requirement for our SSA algorithm
#We print out to see the number of instances in each of the classes
print(f'Training set:{x_train.shape}')
print(f'Testing set:{x_test.shape}')
print(f'Training_Label set:{y_train.shape}')
print(f'Testing_Lable set:{y_test.shape}')

Error rates of the 5 splits [0.04, 0.04, 0.0, 0.0, 0.04]
Index of best split:  2
Shape of selected split:  (253, 300)
Training set:(228, 300)
Testing set:(25, 300)
Training_Label set:(228,)
Testing_Lable set:(25,)


In [45]:
#We start defining our SSA funtions from here

In [46]:
#Random population initailization
def init_position(lb, ub, N, dim):
    X = np.zeros([N, dim], dtype='float')
    for i in range(N):
        for d in range(dim):
            X[i,d] = lb[0,d] + (ub[0,d] - lb[0,d]) * np.random.rand()
            #X[i,d] = lb[d] + (ub[d] - lb[d]) * np.random.rand()
            
    return X

In [47]:
def sigmoid(x):

    return 1 / (1 + np.exp(-x))

def binary_conversion(X, thres, N, dim):

    Xbin = np.zeros([N, dim], dtype='int')
    for i in range(N):
        for d in range(dim):
            if sigmoid(X[i, d]) > thres:
                Xbin[i, d] = 1
            else:
                Xbin[i, d] = 0
    return Xbin

In [48]:
#making our search space to remian between 0 and 1
def boundary(x, lb, ub):
    if x < lb:
        x = lb
    if x > ub:
        x = ub
        
    return x

In [49]:
#Error rate/accuracy
def error_rate(xtrain, ytrain, x, opts):
    #parameters
    k = opts['k']
    
    fold = opts['fold']
    xt = fold['xt']
    yt = fold['yt']
    xv = fold['xv']
    yv = fold['yv']
    
    #Number of instances
    num_train = np.size(xt, 0)
    num_valid = np.size(xv, 0)
    #Define selected features
    xtrain = xt[:, x == 1]
    ytrain = yt.reshape(num_train)
    xvalid = xv[:, x == 1]
    yvalid = yv.reshape(num_valid)
    
    #training our KNN model
    mdl = KNeighborsClassifier(n_neighbors=k)
    mdl.fit(xtrain, ytrain)
    
    #making predictions
    ypred = mdl.predict(xvalid)
    acc = accuracy_score(ypred, yvalid)
    #acc = np.sum(yvalid == ypred) / num_valid
    error = 1 - acc
    
    return error

In [50]:
#Error rate and Feature Size (Fitness Fucntion)
def Fun(xtrain, ytrain, x, opts):
    #parameters
    alpha = 0.99 #default value in literature
    beta = 1 - alpha #default value in literature
    
    #original feature size
    max_feat = len(x)
    #number of selected features
    num_feat = np.sum(x==1)
    #If no feature is selected smiles (impossible)
    if num_feat == 0:
        cost = 1 #Add your code if you like
    else:
        #get error rate fucntion
        error = error_rate(xtrain, ytrain, x, opts)
        #objective/fitness fucntion
        cost = alpha * error + beta * (num_feat/max_feat)
    
    return cost

In [51]:
def main_fs(xtrain, ytrain, opts):
    #our parameters
    ub = 1
    lb = 0
    thres = .5
    
    N = opts['N']
    max_iter = opts['T']
    
    #The dimension of our dataset
    dim = np.size(xtrain, 1)
    if np.size(lb) == 1:
        ub = ub * np.ones([1, dim], dtype = 'float')
        lb = lb * np.ones([1, dim], dtype = 'float')
        
    #Initialize position of salps
    X = init_position(lb, ub, N, dim)
    
    fit = np.zeros([N, 1], dtype = 'float')
    Xf = np.zeros([1, dim], dtype = 'float')
    fitF = float('inf')
    curve = np.zeros([1, max_iter], dtype = 'float')
    t = 0
    
    while t < max_iter:
        #binary conversion
        Xbin = binary_conversion(X, thres, N, dim)
        
        #Fitness
        for i in range(N):
            fit[i,0] = Fun(xtrain, ytrain, Xbin[i,:], opts)
            if fit[i,0] < fitF:
                Xf[0,:] = X[i,:]
                fitF = fit[i,0]
                
        #store results
        curve[0,t] = fitF.copy()
        print('Iteration: ', t + 1)
        print("Best (SSA): ", curve[0,t])
        t = t + 1
        
        #Compute coefficent, c1 from SSA algorithm
        c1 = 2 * np.exp(-(4 * t / max_iter)**2)
        
        for i in range(N):
            #First leader update
            if i == 0:
                for d in range(dim):
                    #coefficient of c2 and c3 in the range of 0 - 1
                    c2 = np.random.rand()
                    c3 = np.random.rand()
                    #leader update
                    if c3>=.5:
                        X[i,d] = Xf[0,d] + c1 * ((ub[0,d] -lb[0,d]) * c2 + lb[0,d])
                    else:
                        X[i,d] = Xf[0,d] - c1 * ((ub[0,d] - lb[0,d]) * c2 + lb[0,d])
                        
                    #boundary
                    X[i,d] = boundary(X[i,d], lb[0,d], ub[0,d])
                
            #salp update
            elif i >= 1:
                for d in range(dim):
                    #salp update by following leader salp
                    X[i,d] =(X[i,d] + X[i - 1, d]) / 2
                    #boundary
                    X[i,d] = boundary(X[i,d], lb[0,d], ub[0,d])
                
    
    #Best feature subset
    Gbin = binary_conversion(Xf, thres, 1, dim)
    Gbin = Gbin.reshape(dim)
    pos  = np.asarray(range(0,dim))
    sel_index = pos[Gbin == 1]
    num_feat = len(sel_index)
    
    #create dictionary
    ssa_data = {'sf': sel_index, 'c': curve, 'nf': num_feat}
    
    return ssa_data

In [52]:
#Now you run our SSA algorithm
#Now we set the general parameters
K = 5 #K value in KNN
N = 30 #Population size/swarm size
T = 60 #Max iteration/generations
opts = {'k': K, 'fold': fold, 'N': N, 'T': T}

In [53]:
#Now we perform our feature selection by calling the main module
feature_selection_mdl = main_fs(selected_Top_data, target, opts) #feature selection model
sf = feature_selection_mdl['sf'] #index of selected features
#print(sf)

Iteration:  1
Best (SSA):  0.010000000000000009
Iteration:  2
Best (SSA):  0.006100000000000006
Iteration:  3
Best (SSA):  0.006100000000000006
Iteration:  4
Best (SSA):  0.006100000000000006
Iteration:  5
Best (SSA):  0.006100000000000006
Iteration:  6
Best (SSA):  0.006100000000000006
Iteration:  7
Best (SSA):  0.006100000000000006
Iteration:  8
Best (SSA):  0.006100000000000006
Iteration:  9
Best (SSA):  0.006100000000000006
Iteration:  10
Best (SSA):  0.006100000000000006
Iteration:  11
Best (SSA):  0.006100000000000006
Iteration:  12
Best (SSA):  0.006100000000000006
Iteration:  13
Best (SSA):  0.006100000000000006
Iteration:  14
Best (SSA):  0.006100000000000006
Iteration:  15
Best (SSA):  0.006100000000000006
Iteration:  16
Best (SSA):  0.006100000000000006
Iteration:  17
Best (SSA):  0.006100000000000006
Iteration:  18
Best (SSA):  0.006100000000000006
Iteration:  19
Best (SSA):  0.006100000000000006
Iteration:  20
Best (SSA):  0.006100000000000006
Iteration:  21
Best (SSA):  0

In [54]:
#Model with selected features
num_train = np.size(x_train,0)#checks the number of samples slected in the x_train and assign to num_train
num_test = np.size(x_test, 0)#checks the number of samples slected in the x_train and assign to num_train
xtrain = x_train[:,sf]
ytrain = y_train.reshape(num_train)
xtest = x_test[:,sf]
ytest = y_test.reshape(num_test)

In [55]:
#Now we build our model i.e., the classifier
KNN_model = KNeighborsClassifier(n_neighbors = 5)
KNN_model.fit(xtrain, ytrain)  

KNeighborsClassifier()

In [56]:
#Method to calculate accuracy for our unseen data
def train_test_accuracy(model, xtest, ytest, name = ''):
    y_pred = model.predict(xtest)
    test_accuracy = round(accuracy_score(y_pred, ytest)*100, 2)
    test_F1_measure = round(f1_score(y_pred, ytest)*100,2)
    error = 1 - round(accuracy_score(y_pred, ytest), 3)
        
    return print("The ACCURACY SCORE is: {} \nThe F1_MEASURE is {} \nThe error rate is: {}". format(test_accuracy, test_F1_measure, error))

In [57]:
#Calling our fucntion to display result
train_test_accuracy(KNN_model, xtest, ytest, name = 'SSA')

The ACCURACY SCORE is: 100.0 
The F1_MEASURE is 100.0 
The error rate is: 0.0


In [58]:
#Now we check the number of selected features
Number_of_features = len(sf)
print(f'The number of FEATURES selected is: {Number_of_features}')

The number of FEATURES selected is: 183


#### End of project